# I. SETUP UND DATENSTRUKTUR

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# --- SETS (Mengen) ---
S = ['S1', 'S2', 'S3']                   # S: Menge der Stationen (Bahnhöfe), indiziert durch i
A = ['B1', 'B2', 'B3']                   # A: Menge der Blöcke, indiziert durch a
K = ['Shipment_A', 'Shipment_B']         # K: Menge der Sendungen (O-D-Paare), indiziert durch k

# Qk: Menge der Kandidaten-Pfade q, assoziiert mit Sendung k
Qk = {
    'Shipment_A': ['P1', 'P2'],          # Pfade P1, P2 für Sendung A
    'Shipment_B': ['P3']                 # Pfad P3 für Sendung B
}

# II. PARAMETER LADEN 

Notationen analog dem Paper von Hasany, Shafahi (2018)

Im Folgenden die Parameter für ein Minimalbeispiel mit 3 Stationen, 3 Blocks und 3 Shipments

In [ ]:
# Skalare Parameter
PHI = 500                   # Weight factor used to convert the vehicle running time to a monetary value ($/min/car)
PH = 20160                  # Given planning horizon (min)
B = 1100000000              # A sufficiently large number calculated from the given data
EPSILON = 0.001             # A sufficiently small number calculated from the given data
R = 10                      # Maximum number of trains that can move through each block 

# --- Hilfsmengen für die Indizierung ---
R_set = range(1, R + 2)                         # R_set: binäre Kodierung von n_a (r=1 bis R+1, für n_a=0 bis R)                  
KPQ = [(k, q) for k in K for q in Qk[k]]        # Alle gültigen (Sendung, Pfad) Paare (k, q)

# Block-spezifische Parameter
t_a = {'B1': 500, 'B2': 400, 'B3': 700}                     # Traveling time through block a ∈ A (min)
w_a = {'B1': 60, 'B2': 90, 'B3': 50}                        # Time required to separate and sort each car inside station i ∈ S , which is the starting station of block a ∈ A (min)
chi_a = {'B1': 10000, 'B2': 5000, 'B3': 20000}              # Fixed operating cost per each train moving on block a ∈ A ($/train)
u_a = {'B1': 100, 'B2': 100, 'B3': 100}                     # Pulling capacity of each train moving through block a ∈ A (car/train)

# Stations-spezifische Parameter
V_i = {'S1': 5000, 'S2': 3000, 'S3': 4000}                  # Maximum number of cars handled inside station i ∈ S (car)
N_i = {'S1': 20, 'S2': 15, 'S3': 10}                        # Maximum number of trains that can be dispatched from station i ∈ S (train)

# Sendungs-spezifische Parameter
d_k = {'Shipment_A': 500, 'Shipment_B': 300}                # Demand associated with shipment k ∈ K during the planning horizon (car)
T_k = {'Shipment_A': 10000, 'Shipment_B': 10000}            # Maximum allowable travel time associated with shipment k ∈ K (min)

# --- Binäre Parameter (Relationen) ---
xi_ia = {
    ('S1', 'B1'): 1, ('S1', 'B2'): 0, ('S1', 'B3'): 1,
    ('S2', 'B1'): 0, ('S2', 'B2'): 1, ('S2', 'B3'): 0,
    ('S3', 'B1'): 0, ('S3', 'B2'): 0, ('S3', 'B3'): 0,
}                                                                                                       # 1 if station i ∈ S is the starting station of block a ∈ A and 0 otherwise
delta_kqa = {
    ('Shipment_A', 'P1', 'B1'): 1, ('Shipment_A', 'P1', 'B2'): 1, ('Shipment_A', 'P1', 'B3'): 0,
    ('Shipment_A', 'P2', 'B1'): 0, ('Shipment_A', 'P2', 'B2'): 0, ('Shipment_A', 'P2', 'B3'): 1,
    ('Shipment_B', 'P3', 'B1'): 0, ('Shipment_B', 'P3', 'B2'): 1, ('Shipment_B', 'P3', 'B3'): 0,
}                                                                                                       # 1 if block a ∈ A is on the blocking path q ∈ Q k and 0 otherwise

# --- Abgeleitete Parameter ---

C_kq = {}                                                      #Time spent for classifying inside the yards on blocking path q ∈ Q k 
for k, q in KPQ:
    C_kq[(k, q)] = sum( (t_a[a] + w_a[a]) * delta_kqa[(k, q, a)] for a in A if delta_kqa[(k, q, a)] == 1 )


# f_r: Wert für Linearisierung der Wartezeit 1/(n_a + epsilon)
f_r = {r: 1.0 / ((r - 1) + EPSILON) for r in R_set}

# III. MODELL-DEFINITION

In [ ]:
model = gp.Model("Railroad_Blocking_Problem_MIP")

# ENTSCHEIDUNGSVARIABLEN

# 3.1. Fluss- und Pfadvariablen
# v[k, q]: Wagenfluss (Car flows) auf Pfad q für Sendung k (Continuous, v_q^k)
v = model.addVars(KPQ, name="Flow", vtype=GRB.CONTINUOUS, lb=0)

# x[k, q]: Auswahl des Pfades (1=gewählt, 0=nicht gewählt) für Sendung k (Binary, x_q^k)
x = model.addVars(KPQ, name="Path_Select", vtype=GRB.BINARY)

# 3.2. Blockvariablen
# z[a]: Auswahl des Blocks (1=gewählt, 0=nicht gewählt) (Binary, z_a)
z = model.addVars(A, name="Block_Select", vtype=GRB.BINARY)

# bn[a, r]: Binäre Variable zur Kodierung der Anzahl Züge n_a. 
# 1, wenn n_a = r-1 (z.B. r=1 => n_a=0; r=2 => n_a=1) (Binary, bn_r^a)
bn = model.addVars(A, R_set, name="Train_Number_Binary", vtype=GRB.BINARY)


# IV. ZIELFUNKTION

In [ ]:
# Shipping Cost: Monatärer Wert der Zeit (Reisezeit + Klassifizierung)
# Sum_{k in K} Sum_{q in Q^k} PHI * C_q^k * v_q^k
shipping_cost = gp.quicksum(PHI * C_kq[k, q] * v[k, q] for k, q in KPQ)

# Operating Cost: Fixkosten für das Betreiben von Zügen
# Sum_{a in A} chi_a * n_a
# n_a ist durch die binäre Summe ersetzt: Sum_{r=1}^{R+1} (r-1) * bn_r^a
operating_cost = gp.quicksum(chi_a[a] * gp.quicksum((r - 1) * bn[a, r] for r in R_set) for a in A)

# Minimierung der Gesamtkosten
model.setObjective(shipping_cost + operating_cost, GRB.MINIMIZE)

# V. NEBENBEDINGUNGEN (CONSTRAINTS)


In [ ]:
# 5.1. Block-Pfad-Konsistenz (Gleichung (2))
# Sum_{k,q} delta_a^q * x_q^k <= (Sum_{k,q} delta_a^q) * z_a
# Wenn ein Pfad x_q^k (der Block a nutzt) gewählt wird, MUSS Block z_a=1 sein.
model.addConstrs(
    (gp.quicksum(delta_kqa[k, q, a] * x[k, q] for k, q in KPQ if delta_kqa[k, q, a] == 1) <=
     gp.quicksum(delta_kqa[k, q, a] for k, q in KPQ if delta_kqa[k, q, a] == 1) * z[a]
     for a in A), name="Consist_BlockPath(2)"
)

In [ ]:
# 5.2. Nachfrageerfüllung (Gleichung (3))
# Sum_{q in Q^k} v_q^k = d^k
# Der gesamte Fluss für jede Sendung k muss die Nachfrage d^k erfüllen.
model.addConstrs(
    (gp.quicksum(v[k, q] for q in Qk[k]) == d_k[k]
     for k in K), name="Demand_Fullfillment(3)"
)

In [ ]:
# 5.3. Wagen-Kapazität (Gleichung (4))
# Sum_{a in A} Sum_{k,q} xi_i^a * delta_a^q * v_q^k <= V(i)
# Die Gesamtzahl der abgehenden Wagen (über alle Blöcke a, die in i beginnen) darf V(i) nicht überschreiten.
model.addConstrs(
    (gp.quicksum(xi_ia[i, a] * delta_kqa[k, q, a] * v[k, q] 
                 for a in A for k, q in KPQ if xi_ia[i, a] == 1 and delta_kqa[k, q, a] == 1) 
     <= V_i[i]
     for i in S), name="Car_Capacity(4)"
)

In [ ]:
# 5.4. Pfad-Fluss-Verknüpfung (Gleichung (5))
# v_q^k <= d^k * x_q^k
# Verknüpft den kontinuierlichen Fluss v mit der binären Auswahl x: Fluss > 0 nur wenn Pfad x=1.
model.addConstrs(
    (v[k, q] <= d_k[k] * x[k, q]
     for k, q in KPQ), name=f"Path_Flow_Link(5)[{k}]"
)

In [ ]:
# 5.5. Zug-Wagen-Kapazität (Linearisierte Gleichung (18), ersetzt (6))
# Sum_{k,q} delta_a^q * v_q^k <= u_a * n_a
# Der Wagenfluss durch Block a ist begrenzt durch die Zugkapazität (u_a) multipliziert mit der Anzahl der Züge (n_a).
model.addConstrs(
    (gp.quicksum(delta_kqa[k, q, a] * v[k, q] for k, q in KPQ if delta_kqa[k, q, a] == 1) 
     <= u_a[a] * gp.quicksum((r - 1) * bn[a, r] for r in R_set)
     for a in A), name="Train_Car_Capacity_Lin(18)"
)

In [ ]:
# 5.6. Block-Zug-Existenz (Linearisierte Gleichung (19), ersetzt (7))
# n_a >= z_a
# Wenn Block a gewählt (z_a=1), muss mindestens ein Zug (n_a >= 1) zugewiesen werden.
model.addConstrs(
    (gp.quicksum((r - 1) * bn[a, r] for r in R_set) >= z[a]
     for a in A), name="Block_Train_Exist_Lin(19)"
)

In [ ]:
# 5.7. Zug-Abfahrts-Kapazität (Linearisierte Gleichung (20), ersetzt (8))
# Sum_{a in A} xi_i^a * n_a <= N(i)
# Die Gesamtzahl der abfahrenden Züge (n_a) von Station i darf N(i) nicht überschreiten.
model.addConstrs(
    (gp.quicksum(xi_ia[i, a] * (r - 1) * bn[a, r] for a in A for r in R_set if xi_ia[i, a] == 1) 
     <= N_i[i]
     for i in S), name="Train_Dispatch_Capacity_Lin(20)"
)

In [ ]:
# 5.8. Maximale Reisezeit (Linearisierte Gleichung (21), ersetzt (9))
# C_q^k + Sum_{a in A} (PH/2) * delta_a^q * (1 / (n_a + epsilon)) <= T^k + B * (1 - x_q^k)
# Die Reisezeit (Klassifizierung + Wartezeit) muss unter dem Limit T^k liegen, wenn der Pfad x_q^k=1 gewählt wird.
# Der nichtlineare Term (1 / (n_a + epsilon)) wird durch Sum_{r} f_r * bn_r^a ersetzt.
model.addConstrs(
    (C_kq[k, q] + gp.quicksum((PH / 2.0) * delta_kqa[k, q, a] * gp.quicksum(f_r[r] * bn[a, r] for r in R_set) 
                             for a in A if delta_kqa[k, q, a] == 1)
     <= T_k[k] + B * (1 - x[k, q])
     for k, q in KPQ), name="Max_Travel_Time_Lin(21)"
)

In [ ]:
# 5.9. Binäre Auswahl für n_a (Gleichung (15))
# Sum_{r=1}^{R+1} bn_r^a = 1
# Stellt sicher, dass für jeden Block a genau ein Wert für n_a (d.h. genau ein bn_r^a) ausgewählt wird.
model.addConstrs(
    (gp.quicksum(bn[a, r] for r in R_set) == 1
     for a in A), name="Train_Number_Binary_Select(15)"
)

# VI. LÖSUNG UND ANALYSE

In [ ]:
# MODELLOPTIMIERUNG 
model.optimize()

# LÖSUNG AUSGEBEN
if model.status == GRB.OPTIMAL:
    print("\n--- Optimale Lösung gefunden ---")
    print(f"Gesamtkosten: {model.objVal:.2f}")

    print("\nAusgewählte Pfade und Flüsse (v_q^k > 0):")
    for k, q in KPQ:
        if v[k, q].X > 0.01: # Zeige nur Flüsse > 0
            print(f"  Sendung {k}, Pfad {q}: Fluss = {v[k, q].X:.0f} Wagen")

    print("\nAusgewählte Blöcke und Zuganzahl (n_a > 0):")
    for a in A:
        # Berechne n_a aus den bn_r^a Variablen
        n_a_val = 0
        for r in R_set:
            if bn[a, r].X > 0.9: # Wenn bn_r^a = 1
                n_a_val = r - 1
                break
        
        if n_a_val > 0:
            print(f"  Block {a}: {n_a_val} Züge")

elif model.status == GRB.INFEASIBLE:
    print("\nModell ist unlösbar.")
    
    # # IIS DIAGNOSE 
    # model.computeIIS()
    # model.write("model_iis.ilp")

else:
    print(f"\nOptimierung beendet mit Status: {model.status}")

# Modelldatei speichern
model.write("railroad_blocking_model.lp")